# Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
import gc  # Para coleta de lixo explícita
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [ ]:
sales_df = pd.read_csv("product_structures_sales.csv")
prices_df = pd.read_csv("product_prices_leaflets.csv")
campaigns_df = pd.read_csv("chain_campaigns.csv")

In [3]:
sales_df

,structure_level_4,structure_level_3,structure_level_2,structure_level_1,sku,time_key,quantity
0,3020206,30202,302,3,3111,20230618,18.6840
1,3020608,30206,302,3,3278,20240731,396.1008
2,3020809,30208,302,3,3603,20230807,6.2280
3,3020608,30206,302,3,4604,20230131,27.4032
4,3040808,30408,304,3,3041,20230906,6.2280
...,...,...,...,...,...,...,...
1864589,3020809,30208,302,3,1694,20230531,100.8936
1864590,2020604,20206,202,2,3314,20230308,66.0168
1864591,3040601,30406,304,3,2318,20230307,47.3328
1864592,3020401,30204,302,3,4443,20230204,382.3992


# Functions

In [3]:
def prepare_data(sales_df, prices_df, campaigns_df):
    """
    Preparar e integrar os três conjuntos de dados.
    """
    print("Processando e integrando os dados...")
    
    # Converter time_key para formato de data
    sales_df['date'] = pd.to_datetime(sales_df['time_key'].astype(str), format='%Y%m%d')
    prices_df['date'] = pd.to_datetime(prices_df['time_key'].astype(str), format='%Y%m%d')
    
    # Converter datas de campanha para formato datetime
    campaigns_df['start_date'] = pd.to_datetime(campaigns_df['start_date'])
    campaigns_df['end_date'] = pd.to_datetime(campaigns_df['end_date'])
    
    # Calcular o target_price: pvp_was * (1 - discount)
    prices_df['target_price'] = prices_df['pvp_was'] * (1 - prices_df['discount'])
    
    # Preencher valores nulos em leaflet
    prices_df['leaflet'] = prices_df['leaflet'].fillna('none')
    
    return sales_df, prices_df, campaigns_df

def merge_data(sales_df, prices_df, campaigns_df):
    """
    Juntar os dados em um único dataframe para análise e modelagem.
    Versão otimizada para maior desempenho.
    """
    print("Juntando os dados...")
    
    # Selecionar apenas as colunas necessárias para reduzir uso de memória
    df = prices_df[['sku', 'date', 'competitor', 'pvp_was', 'discount', 'flag_promo', 'leaflet', 'target_price']].copy()
    
    # Agregar vendas por sku e data uma única vez
    sales_agg = sales_df.groupby(['sku', 'date'])['quantity'].sum().reset_index()
    
    # Fazer o merge usando um join mais eficiente
    df = pd.merge(df, sales_agg, on=['sku', 'date'], how='left')
    
    # Adicionar informações de estrutura hierárquica do produto
    structure_cols = ['structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4']
    structure_info = sales_df[['sku'] + structure_cols].drop_duplicates().set_index('sku')
    
    # Merge usando indices para maior eficiência
    df = df.merge(structure_info, left_on='sku', right_index=True, how='left')
    
    # Adicionar informações de campanhas de forma vetorizada
    print("Aplicando informações de campanhas (método otimizado)...")
    
    # Inicializar coluna padrão
    df['active_campaign'] = 'no_campaign'
    
    # Criar um dataframe expandido com todas as datas possíveis para campanhas
    campaign_dates = []
    for _, campaign in campaigns_df.iterrows():
        date_range = pd.date_range(start=campaign['start_date'], end=campaign['end_date'])
        for date in date_range:
            campaign_dates.append({
                'date': date,
                'competitor': campaign['competitor'],
                'chain_campaign': campaign['chain_campaign']
            })
    
    if campaign_dates:  # Verificar se a lista não está vazia
        campaign_df = pd.DataFrame(campaign_dates)
        
        # Fazer merge eficiente usando as datas e competidores como chave
        df = pd.merge(
            df,
            campaign_df,
            on=['date', 'competitor'],
            how='left',
            suffixes=('', '_campaign')
        )
        
        # Atualizar a coluna active_campaign onde temos correspondência
        mask = ~df['chain_campaign'].isna()
        df.loc[mask, 'active_campaign'] = df.loc[mask, 'chain_campaign']
        
        # Remover a coluna redundante
        df = df.drop('chain_campaign', axis=1)
    
    # Adicionar recursos temporais de forma vetorizada
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day_of_week'] = df['date'].dt.dayofweek
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    
    # Preencher valores nulos na coluna de quantidade com 0
    df['quantity'] = df['quantity'].fillna(0)
    
    # Liberar memória
    del sales_agg, structure_info
    gc.collect()
    
    return df

def feature_engineering(df):
    """
    Criar recursos adicionais que podem ajudar na previsão de preços.
    Versão otimizada para maior desempenho.
    """
    print("Criando features adicionais...")
    
    # Fazer cálculos estatísticos em um único passo para evitar múltiplos groupby
    print("Calculando estatísticas...")
    
    # Estatísticas por SKU
    sku_stats = df.groupby('sku')['target_price'].agg(['mean', 'std', 'min', 'max']).reset_index()
    sku_stats.columns = ['sku', 'mean_price', 'std_price', 'min_price', 'max_price']
    
    # Média de preço por competitor
    #comp_price = df.groupby('competitor')['target_price'].mean().reset_index()
    #comp_price.columns = ['competitor', 'competitor_avg_price']
    
    # Médias por níveis de estrutura (mais eficiente)
    structure_prices = {}
    for level in ['structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4']:
        level_price = df.groupby(level)['target_price'].mean().reset_index()
        level_price.columns = [level, f'{level}_avg_price']
        structure_prices[level] = level_price
    
    # Fazer merges em sequência é mais eficiente que múltiplos merges independentes
    df = pd.merge(df, sku_stats, on='sku', how='left')
    #df = pd.merge(df, comp_price, on='competitor', how='left')
    
    for level in ['structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4']:
        df = pd.merge(df, structure_prices[level], on=level, how='left')
    
    # Marcar períodos promocionais (baseado em flag_promo)
    df['is_promo_period'] = df['flag_promo'].astype(int)
    
    # Criar indicadores para campanhas e leaflets usando get_dummies (mais eficiente)
    print("Criando indicadores para campanhas e leaflets...")
    campaign_dummies = pd.get_dummies(df['active_campaign'], prefix='campaign')
    leaflet_dummies = pd.get_dummies(df['leaflet'], prefix='leaflet')
    
    # Concatenar de uma vez
    df = pd.concat([df, campaign_dummies, leaflet_dummies], axis=1)
    
    # Liberar memória
    #del sku_stats, comp_price, structure_prices, campaign_dummies, leaflet_dummies
    del sku_stats, structure_prices, campaign_dummies, leaflet_dummies
    gc.collect()
    
    return df

def prepare_train_test(df, target_competitor):
    """
    Preparar conjuntos de treinamento e teste.
    """
    print("Preparando conjuntos de treinamento e teste...")
    
    # Filtrar para competidores alvo
    pred_df = df[df['competitor'] == target_competitor].copy()
    
    # Definir recursos e alvo    
    '''
    features = [
        'sku', 'structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4',
        'year', 'month', 'day_of_week', 'is_weekend', 'is_promo_period',
        'discount', 'flag_promo', 'quantity', 'competitor_avg_price', 'mean_price', 'std_price', 'min_price', 'max_price',
        'structure_level_1_avg_price', 'structure_level_2_avg_price', 'structure_level_3_avg_price', 'structure_level_4_avg_price'        
    ]
    #'''

    features = [
        'sku', 'structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4',
        'year', 'month', 'day_of_week', 'is_weekend', 'is_promo_period',
        'discount', 'flag_promo', 'quantity', 'mean_price', 'std_price', 'min_price', 'max_price',
        'structure_level_1_avg_price', 'structure_level_2_avg_price', 'structure_level_3_avg_price', 'structure_level_4_avg_price'        
    ]

    # Adicionar colunas de campanhas e leaflets
    campaign_cols = [col for col in pred_df.columns if col.startswith('campaign_')]
    leaflet_cols = [col for col in pred_df.columns if col.startswith('leaflet_')]
    features.extend(campaign_cols)
    features.extend(leaflet_cols)

    # Obter X e y
    X = pred_df[features]
    y = pred_df['target_price']

    return X, y, features, pred_df

def build_and_train_model(X, y, df, target_competitor):
    """
    Construir e treinar o modelo de previsão.
    """
    print("Construindo e treinando o modelo...")
    

    # Extract product category for grouped evaluation
    categories = df['structure_level_2']

    # --- 1. Configure TimeSeriesSplit ---

    # Set up time-based cross-validation
    n_samples = len(X)
    max_folds = min(6, n_samples - 1)  # Ensure at least 2 samples per fold
    if max_folds < 2:
        raise ValueError(f"Not enough data for time-based cross-validation ({n_samples} samples).")

    tscv = TimeSeriesSplit(n_splits=max_folds)
    print(f"TimeSeriesSplit with {max_folds} folds on {n_samples} samples")

    # Containers to store results
    mae_scores = []
    category_metrics = []

    for fold, (train_idx, test_idx) in enumerate(tscv.split(X), 1):
        # Split data into training and test sets
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        cat_test = categories.iloc[test_idx]

        # Create LightGBM datasets
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

        # --- Model Parameters (choose one version at a time) ---

        ### Version 1 - Basic tuned GBDT
        #'''
        params = {
            'objective': 'regression',           # Type of task: 'regression' for continuous output prediction
            'metric': 'mae',                     # Metric used to evaluate performance: 'mae' = Mean Absolute Error
            'boosting_type': 'gbdt',             # Boosting algorithm: 'gbdt' = Gradient Boosted Decision Trees

            'num_leaves': 21,                    # Max number of leaves in one tree (controls model complexity)
            'learning_rate': 0.01,               # Shrinks the contribution of each tree (lower = better generalization)
            'max_depth': 6,                      # Max depth of trees (limits overfitting; -1 means no limit)

            'lambda_l1': 0.1,                    # L1 regularization (controls model complexity)
            'lambda_l2': 0.1,                    # L2 regularization (controls model complexity)

            'feature_fraction': 0.9,             # Fraction of features used per iteration (column sampling)
            'bagging_fraction': 0.8,             # Fraction of data used per iteration (row sampling)
            'bagging_freq': 5,                   # Perform bagging every k iterations (0 = disable)

            'verbose': -1,                       # Controls logging: < 0 means no output
            'seed': 42                           # Random seed for reproducibility
        }
        #'''



        # --- Additional useful parameters you can try ---

        # 'min_data_in_leaf': 20,               # Minimum number of samples per leaf (higher reduces overfitting)
        # 'min_gain_to_split': 0.01,            # Minimum loss reduction required to make a further partition
        # 'lambda_l1': 0.1,                     # L1 regularization (controls model complexity)
        # 'lambda_l2': 0.1,                     # L2 regularization (controls model complexity)
        # 'max_bin': 255,                       # Max number of bins used for discretizing continuous features
        # 'early_stopping_round': 50,          # Early stopping based on validation metric (used via callbacks)
        # 'first_metric_only': True,           # Stop training if the first metric does not improve (only use first listed metric)
        # 'drop_rate': 0.1,                     # (For DART) Dropout rate for trees
        # 'skip_drop': 0.5,                     # (For DART) Probability of skipping dropout
        # 'xgboost_dart_mode': False,          # (For DART) Whether to use xgboost's dart mode
        # 'extra_trees': False,                # Whether to use extremely randomized trees (more randomness)
        # 'force_row_wise': False,             # Forces row-wise histogram building (useful for GPU or speed tuning)
        # 'deterministic': True,               # Ensures reproducibility of results (same seed = same result)



        ### Version 2 - GBDT with regularization and deeper trees
        '''
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'gbdt',
            'num_leaves': 63,
            'learning_rate': 0.01,
            'max_depth': 8,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.7,
            'bagging_freq': 4,
            'min_data_in_leaf': 20,
            'lambda_l1': 0.1,
            'lambda_l2': 0.1,
            'verbose': -1,
            'seed': 42,
            'first_metric_only': True
        }
        #'''

        ### Version 3 - Using DART boosting
        '''
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'dart',        # Dropouts meet Multiple Additive Regression Trees
            'num_leaves': 41,
            'learning_rate': 0.03,
            'max_depth': 6,
            'feature_fraction': 0.85,
            'bagging_fraction': 0.85,
            'bagging_freq': 5,
            'min_data_in_leaf': 30,
            'lambda_l1': 0.05,
            'lambda_l2': 0.15,
            'min_gain_to_split': 0.02,
            'drop_rate': 0.1,
            'xgboost_dart_mode': True,          # (For DART) Whether to use xgboost's dart mode
            'max_drop': 50,
            'verbose': -1,
            'seed': 42
        }
        #'''

        # --- Train Model ---
        model = lgb.train(
            params,
            train_data,
            valid_sets=[valid_data],
            num_boost_round=1000,
            callbacks=[
                lgb.early_stopping(stopping_rounds=50, verbose=False),
                lgb.log_evaluation(period=0)
            ]
        )

        # Save model from last fold
        if fold == max_folds:
            model_filename = f"lgbm_{target_competitor}_{datetime.now().strftime('%Y%m%d_%H%M')}.pkl"
            joblib.dump(model, model_filename)
            print(f"\nModel saved as: {model_filename}")

        # --- Evaluate predictions ---
        y_pred = model.predict(X_test, num_iteration=model.best_iteration)
        fold_mae = mean_absolute_error(y_test, y_pred)
        mae_scores.append(fold_mae)
        print(f"Fold {fold} MAE: {fold_mae:.2f}")

        # Create dataframe for category-wise metrics
        df_cat = pd.DataFrame({
            'category': cat_test,
            'y_true': y_test,
            'y_pred': y_pred
        })

        # Function to compute key regression metrics
        def get_metrics(df_cat):
            y_true = df_cat['y_true']
            y_pred = df_cat['y_pred']
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            r2 = r2_score(y_true, y_pred)
            mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100 if np.all(y_true != 0) else np.nan
            return pd.Series({'MAE': mae, 'RMSE': rmse, 'R2': r2, 'MAPE': mape})

        # Store category-level metrics
        cat_metrics = df_cat.groupby('category').apply(get_metrics)
        category_metrics.append(cat_metrics)
    
    return model, category_metrics, mae_scores

def evaluate_model(category_metrics, df):
    """
    Avaliar o desempenho do modelo.
    """
    print("Avaliando o modelo...")
    
    # Combine metrics from all folds into a single dataframe
    df_all_metrics = pd.concat(category_metrics, keys=range(1, len(category_metrics) + 1), names=['Fold', 'Category'])
    mean_metrics_per_cat = df_all_metrics.groupby('Category').mean()

    # --- 3. Final Summary ---

    # Global MAE statistics
    mae_scores = [cat['MAE'].mean() for cat in category_metrics]
    print(f"\nAverage MAE across folds: {np.mean(mae_scores):.2f}, Std MAE: {np.std(mae_scores):.2f}")

    # Category-wise MAE statistics with stability check
    category_mae_df = pd.DataFrame({f'Fold_{i+1}': cat['MAE'] for i, cat in enumerate(category_metrics)})
    category_mae_df['Mean_MAE'] = category_mae_df.mean(axis=1)
    category_mae_df['Std_MAE'] = category_mae_df.std(axis=1)
    category_mae_df['Stability_Ratio'] = category_mae_df['Std_MAE'] / category_mae_df['Mean_MAE']

    print("\nCategory-wise MAE performance:")
    print(category_mae_df[['Mean_MAE', 'Std_MAE', 'Stability_Ratio']].sort_values('Mean_MAE'))

    # Stability threshold check
    max_ratio = category_mae_df['Stability_Ratio'].max()
    print(f"\nMax Stability Ratio: {max_ratio:.2%}")
    if max_ratio < 0.10:
        print("SUCCESS: Variation across folds is under 10%")
    else:
        print("WARNING: Some categories exceed 10% variation across folds")

    # --- 4. HTML Summary Table ---

    # Statistics for the actual target prices by category
    target_stats = df.groupby('structure_level_2')['target_price'].agg(['mean', 'median', 'std'])
    target_stats.columns = ['Mean_Target_Price', 'Median_Target_Price', 'Std_Target_Price']

    # Merge prediction errors with target stats
    category_summary = category_mae_df[['Mean_MAE', 'Std_MAE', 'Stability_Ratio']].join(target_stats)

    # Relative error comparison
    category_summary['MAE_%_of_Mean'] = category_summary['Mean_MAE'] / category_summary['Mean_Target_Price']
    category_summary['MAE_%_of_Median'] = category_summary['Mean_MAE'] / category_summary['Median_Target_Price']

    # Sort by relative MAE
    category_summary_sorted = category_summary.sort_values('MAE_%_of_Mean')

    # Display in notebook (HTML)
    from IPython.display import display, HTML
    print("\nCategory-level model performance compared to target_price statistics:")
    display(HTML(category_summary_sorted[['Mean_MAE', 'Std_MAE',
                                        'Mean_Target_Price', 'Median_Target_Price',
                                        'MAE_%_of_Mean', 'MAE_%_of_Median',
                                        'Stability_Ratio']].to_html(float_format="%.2f")))

def predict_price(sku, date, competitor, sales_df_clean, prices_df_clean, campaigns_df_clean, features, model):
    """
    Fazer uma previsão para um novo caso.
    """
    if sku not in sales_df_clean['sku'].unique():
        return

    print("Create missing data to make predictions...")
    
    # Extrair informações estruturais do SKU
    sku_info = sales_df_clean[sales_df_clean['sku'] == sku][['structure_level_1', 'structure_level_2', 
                                            'structure_level_3', 'structure_level_4']].iloc[0]
    
    # Converter date para datetime se for string
    if isinstance(date, str):
        date = pd.to_datetime(date)
        
    # Criar dataframe para este caso específico com valores padrão
    new_data = {
        'sku': sku,
        'date': date,
        'competitor': competitor,
        'structure_level_1': sku_info['structure_level_1'],
        'structure_level_2': sku_info['structure_level_2'],
        'structure_level_3': sku_info['structure_level_3'],
        'structure_level_4': sku_info['structure_level_4'],
        'year': date.year,
        'month': date.month,
        'day_of_week': date.dayofweek,
        'is_weekend': 1 if date.dayofweek in [5, 6] else 0,
        'is_promo_period': 0,  # Valor padrão
    }
    
    print("Check campains.")
    # Verificar campanhas ativas na data
    active_campaign = 'no_campaign'
    for _, campaign in campaigns_df_clean[campaigns_df_clean['competitor'] == competitor].iterrows():
        if campaign['start_date'] <= date <= campaign['end_date']:
            active_campaign = campaign['chain_campaign']
            break
    
    print("Check active_campaign.")
    # Preencher valores para campanhas
    for feature in features:
        if feature.startswith('campaign_'):
            campaign_name = feature.replace('campaign_', '')
            new_data[feature] = 1 if active_campaign == campaign_name or feature == '' else 0
    
    print("Check leaflets.")
    # Preencher valores para leaflets
    for feature in features:
        if feature.startswith('leaflet_'):
            leaflet_name = feature.replace('leaflet_', '')
            new_data[feature] = 1 if leaflet_name == 'none' else 0

    print(f'Check mean, std, min and max prices for sku.')
    # Para valores estatísticos, usar médias do dataframe completo
    # Esta é uma simplificação; em produção, você armazenaria estas estatísticas
    prices_stats = prices_df_clean[prices_df_clean['sku'] == sku]['target_price']
    if len(prices_stats) > 0:
        new_data['mean_price'] = prices_stats.mean()
        new_data['std_price'] = prices_stats.std() if len(prices_stats) > 1 else 0
        new_data['min_price'] = prices_stats.min()
        new_data['max_price'] = prices_stats.max()

    # Médias por nível estrutural
    for level in ['structure_level_1', 'structure_level_2', 'structure_level_3', 'structure_level_4']:
        # Obter os valores únicos do nível atual para o SKU específico
        list_structure_values = sales_df_clean[sales_df_clean['sku'] == sku][level].unique()
        
        # Filtrar preços para SKUs que têm o mesmo valor de estrutura
        filtered_prices = prices_df_clean[prices_df_clean['sku'].isin(list_structure_values)]
        
        # Calcular o preço médio
        avg_price = filtered_prices['target_price'].mean()
        
        # Atribuir o preço médio à coluna correspondente
        new_data[f'{level}_avg_price'] = avg_price
  
    print('Check competitor_avg_price.')
    # Média de preço do competidor
    '''
    comp_prices = prices_df[prices_df['competitor'] == competitor]['target_price']
    new_data['competitor_avg_price'] = comp_prices.mean() #if len(comp_prices) > 0 else new_data['mean_price']
    #'''

    # Criar dataframe
    new_case = pd.DataFrame([new_data])
    
    # Selecionar apenas as features usadas no modelo
    available_features = [f for f in features if f in new_case.columns]
    X_pred = new_case[available_features]
    
    # Verificar quais features estão faltando e adicionar valores padrão
    missing_features = set(features) - set(available_features)
    for feature in missing_features:
        if feature.startswith('campaign_'):
            X_pred[feature] = 0
        elif feature.startswith('leaflet_'):
            X_pred[feature] = 0 if feature != 'leaflet_none' else 1
        else:
            # Para outras features, usar a média global
            X_pred[feature] = 0
    
    print("Predicting target_price.")
    
    # Fazer a previsão
    predicted_price = model.predict(X_pred)[0]
    
    return predicted_price


# Prepare and Merge Data

In [4]:
# Preparar e integrar os dados
#sales_df, prices_df, campaigns_df = prepare_data(sales_df, prices_df, campaigns_df)
sales_df_clean, prices_df_clean, campaigns_df_clean = prepare_data(sales_df, prices_df, campaigns_df)

sales_df_clean.to_parquet("sales_df_clean.parquet")
prices_df_clean.to_parquet("prices_df_clean.parquet")
campaigns_df_clean.to_parquet("campaigns_df_clean.parquet")


#df = merge_data(sales_df, prices_df, campaigns_df)
df = merge_data(sales_df_clean, prices_df_clean, campaigns_df_clean)
df = feature_engineering(df)

df.to_parquet("df.parquet")

df.dtypes

Processando e integrando os dados...
Juntando os dados...
Aplicando informações de campanhas (método otimizado)...
Criando features adicionais...
Calculando estatísticas...
Criando indicadores para campanhas e leaflets...


sku                                     int64
date                           datetime64[ns]
competitor                             object
pvp_was                               float64
discount                              float64
flag_promo                              int64
leaflet                                object
target_price                          float64
quantity                              float64
structure_level_1                     float64
structure_level_2                     float64
structure_level_3                     float64
structure_level_4                     float64
active_campaign                        object
year                                    int32
month                                   int32
day_of_week                             int32
is_weekend                              int32
mean_price                            float64
std_price                             float64
min_price                             float64
max_price                         

# Competitor Data

In [5]:
'''
target_competitor = 'competitorA'

X, y, features, df_model = prepare_train_test(df,target_competitor)
model, category_metrics, mae_scores = build_and_train_model(X, y, df_model, target_competitor)
#'''

"\ntarget_competitor = 'competitorA'\n\nX, y, features, df_model = prepare_train_test(df,target_competitor)\nmodel, category_metrics, mae_scores = build_and_train_model(X, y, df_model, target_competitor)\n#"

# Model

In [6]:
'''
model, category_metrics, mae_scores = build_and_train_model(X, y, df_model, target_competitor)
evaluate_model(category_metrics, df_model)
#'''

'\nmodel, category_metrics, mae_scores = build_and_train_model(X, y, df_model, target_competitor)\nevaluate_model(category_metrics, df_model)\n#'

# Pipeline Data and Model

In [ ]:
import joblib

df = pd.read_parquet("df.parquet")

model_registry = {}

for comp in df['competitor'].unique():
    if comp == 'chain':
        continue
    
    X, y, features, df_model = prepare_train_test(df, comp)
    model, category_metrics, mae_scores = build_and_train_model(X, y, df_model, comp)

    model_registry[comp] = {
        #"X": X,
        #"y": y,
        "features": features,
        "df_model": df_model,
        "model": model,
        "columns": list(X.columns),
        "dtypes": X.dtypes.to_dict(),  # guarda os tipos para validação na API        
        "metrics": category_metrics,
        "mae": mae_scores
    }

# Salvar tudo num arquivo
joblib.dump(model_registry, "models_registry.pkl")

Preparando conjuntos de treinamento e teste...
Construindo e treinando o modelo...
TimeSeriesSplit with 6 folds on 1099604 samples
Fold 1 MAE: 2.22
Fold 2 MAE: 2.22
Fold 3 MAE: 2.19
Fold 4 MAE: 2.19
Fold 5 MAE: 2.19

Model saved as: lgbm_competitorA_20250520_2004.pkl
Fold 6 MAE: 2.20
Preparando conjuntos de treinamento e teste...
Construindo e treinando o modelo...
TimeSeriesSplit with 6 folds on 268763 samples
Fold 1 MAE: 2.21
Fold 2 MAE: 2.14
Fold 3 MAE: 2.16
Fold 4 MAE: 2.15
Fold 5 MAE: 2.17

Model saved as: lgbm_competitorB_20250520_2005.pkl
Fold 6 MAE: 2.10


['models_registry.pkl']

# Predictions

In [4]:
sales_df_clean = pd.read_parquet("sales_df_clean.parquet")
prices_df_clean  = pd.read_parquet("prices_df_clean.parquet")
campaigns_df_clean  = pd.read_parquet("campaigns_df_clean.parquet")

In [5]:
sales_df_clean

,structure_level_4,structure_level_3,structure_level_2,structure_level_1,sku,time_key,quantity,date
0,3020206,30202,302,3,3111,20230618,18.6840,2023-06-18
1,3020608,30206,302,3,3278,20240731,396.1008,2024-07-31
2,3020809,30208,302,3,3603,20230807,6.2280,2023-08-07
3,3020608,30206,302,3,4604,20230131,27.4032,2023-01-31
4,3040808,30408,304,3,3041,20230906,6.2280,2023-09-06
...,...,...,...,...,...,...,...,...
1864589,3020809,30208,302,3,1694,20230531,100.8936,2023-05-31
1864590,2020604,20206,202,2,3314,20230308,66.0168,2023-03-08
1864591,3040601,30406,304,3,2318,20230307,47.3328,2023-03-07
1864592,3020401,30204,302,3,4443,20230204,382.3992,2023-02-04


In [6]:
print(sales_df_clean['sku'].dtype)
print(sales_df_clean['sku'].unique()[:5])

int64
[3111 3278 3603 4604 3041]


In [7]:
print(sales_df_clean['sku'].unique())


[3111 3278 3603 ... 3543 2593 3663]


In [8]:
# Carregar os dados e modelos
model_registry = joblib.load("models_registry.pkl")

sales_df_clean = pd.read_parquet("sales_df_clean.parquet")
prices_df_clean  = pd.read_parquet("prices_df_clean.parquet")
campaigns_df_clean  = pd.read_parquet("campaigns_df_clean.parquet")

# Fazer previsões para novos casos"
sku = 3111
date = pd.to_datetime('2025-06-30')
competitor = 'competitorA'

if competitor == 'competitorA':
    model = model_registry[competitor]["model"]
    features = model_registry[competitor]["features"]
    
    predicted_price = predict_price(sku, date, competitor, sales_df_clean, prices_df_clean, campaigns_df_clean, features, model)

elif competitor == 'competitorB':
    model = model_registry[competitor]["model"]
    features = model_registry[competitor]["features"]
    
    predicted_price = predict_price(sku, date, competitor, sales_df_clean, prices_df_clean, campaigns_df_clean, features, model)

print(f"Preço previsto para SKU {sku}, data {date}, competidor {competitor}: {predicted_price:.2f}")

Create missing data to make predictions...
Check campains.
Check active_campaign.
Check leaflets.
Check mean, std, min and max prices for sku.
Check competitor_avg_price.
Predicting target_price.
Preço previsto para SKU 3111, data 2025-06-30 00:00:00, competidor competitorA: 48.23
